# Move Bounds

Notebook on moving bounds for systems, while also calculating the the $\sigma$s

In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.mixed_system import MixedSystem
import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt
import scipy.linalg 
import scipy.stats 
import tvsclib.utils as utils

from tvsclib.transformations.output_normal import OutputNormal
from tvsclib.transformations.input_normal import InputNormal
from tvsclib.transformations.reduction import Reduction

from structurednets.asset_helpers import get_animal_classes_filepath
from structurednets.extract_features import get_required_indices
from structurednets.models.visionmodel import VisionModel
from structurednets.models.alexnet import AlexNet
from structurednets.models.googlenet import GoogleNet
from structurednets.models.inceptionv3 import InceptionV3
from structurednets.models.mobilenetv2 import MobilenetV2
from structurednets.models.resnet18 import Resnet18
from structurednets.models.vgg16 import VGG16

In [ ]:
import move

## Create a tialor made matrix

Create mateices with combinations of orthogonal vectors such that we can create rank1 block matrices and combine these.

In [ ]:
dims_in =  np.array([6, 3, 5, 2])*3
dims_out = np.array([2, 5, 3, 6])*3

#create orthogonal vectors and normalize them to the size of the matix (i.e. norm(block)/size(block) = const
#Us =np.vstack([np.linalg.svd(np.random.rand(dims_out[i],dims_in[i]))[0][:,1:4]*dims_out[i] for i in range(len(dims_in))])
#Vts=np.hstack([np.linalg.svd(np.random.rand(dims_out[i],dims_in[i]))[2][1:4,:]*dims_in[i] for i in range(len(dims_in))])

#create orthogonal vectors and normalize them to the size of the matix (i.e. norm(block)/size(block) = const
Us =np.vstack([scipy.stats.ortho_group.rvs(dims_out[i])[:,:3]*dims_out[i] for i in range(len(dims_in))])
Vts=np.hstack([scipy.stats.ortho_group.rvs(dims_in[i])[:3,:]*dims_in[i] for i in range(len(dims_in))])



lower = Us[:,:1]@Vts[:1,:]
diag = Us[:,1:2]@Vts[1:2,:]
upper = Us[:,2:3]@Vts[2:3,:]
matrix = np.zeros_like(diag)
a=0;b=0
for i in range(len(dims_in)):
    matrix[a:a+dims_out[i],:b]            =lower[a:a+dims_out[i],:b]
    matrix[a:a+dims_out[i],b:b+dims_in[i]]=diag[a:a+dims_out[i],b:b+dims_in[i]]
    matrix[a:a+dims_out[i],b+dims_in[i]:] =upper[a:a+dims_out[i],b+dims_in[i]:]
    a+=dims_out[i];b+=dims_in[i]
plt.figure()
plt.matshow(matrix)

In [ ]:

a=0;b=0
for i in range(len(dims_in)):
    print(np.linalg.norm(diag[a:a+dims_out[i],b:b+dims_in[i]])/(dims_in[i]*dims_out[i]))
    a+=dims_out[i];b+=dims_in[i]

Create a system agnostic of the state boundareis but with the correct number of states

In [ ]:
dims_in_start = [sum(dims_in)//4]*4
dims_out_start = [sum(dims_out)//4]*4
T = ToeplitzOperator(matrix, dims_in_start, dims_out_start)
S = SystemIdentificationSVD(T,epsilon=1e-12)
system = MixedSystem(S)
utils.show_system(system)

In [ ]:
def cost_error_cut_rel(s,shape,s_a,shape_a):
    tau = 0.9
    c = np.sum(s[s<tau*np.max(s)]**2)/(shape[0]*shape[1])+\
        np.sum(s_a[s_a<tau*np.max(s_a)]**2)/(shape_a[0]*shape_a[1])
    #print("s:",s[s<tau*np.max(s)]," s_a:",s_a[s_a<tau*np.max(s_a)])
    return c


sys_move,input_dims,output_dims = move.move(system,1,cost_error_cut_rel,n_in=4,n_out=4)
sys_move,input_dims,output_dims = move.move(sys_move,1,cost_error_cut_rel,n_in=2,n_out=2)
sys_move,input_dims,output_dims = move.move(sys_move,1,cost_error_cut_rel,n_in=1,n_out=1)
#sys_move,input_dims,output_dims = move.move(sys_move,4,cost_error_cut_abs,n_in=1,n_out=1)


utils.show_system(sys_move)

print(dims_in)
print(np.array(sys_move.dims_in))
print(dims_out)
print(np.array(sys_move.dims_out))


## Test Weight matrix

In [ ]:
#get matrix

model_class = AlexNet
label_filepath = get_animal_classes_filepath()
required_indices = get_required_indices(label_filepath)
model = model_class(required_indices)
output_mat = model.get_optimization_matrix().detach().numpy()



In [ ]:
stages = 20

#set the dims
d_in = output_mat.shape[1]
boundaries = d_in/stages*np.arange(stages+1)
boundaries = np.round(boundaries).astype(int)
dims_in = boundaries[1:]-boundaries[:-1]

d_out = output_mat.shape[0]
boundaries = d_out/stages*np.arange(stages+1)
boundaries = np.round(boundaries).astype(int)
dims_out = boundaries[1:]-boundaries[:-1]

assert sum(dims_in)==d_in and sum(dims_out)==d_out

T = ToeplitzOperator(output_mat, dims_in,dims_out)
S = SystemIdentificationSVD(T,epsilon=1e-16)

system = MixedSystem(S)
#approx =Approximation(system)
print(system)

In [ ]:


def cost(s,shape,s_a,shape_a):
    c =np.sum(s)+np.sum(s_a)
    return c

#cost relative to size
def cost_rel(s,shape,s_a,shape_a):
    print("shape:",shape)
    print("shape_a:",shape_a)
    c =np.sum(s**2)/(shape[0]*shape[1])+np.sum(s_a**2)/(shape_a[0]*shape_a[1])
    return c
  
def cost_check(s,shape,s_a,shape_a):
    print(shape[0]+shape_a[0])
    print(shape[1]+shape_a[1])
    return 0
    

def cost_scaled(s,shape,s_a,shape_a):
    c =np.sum(s/np.max(s))+np.sum(s_a/np.max(s_a))
    return c

def cost_error_cut_rel(s,shape,s_a,shape_a):
    tau = 0.5
    c = np.sum(s[s<tau*np.max(s)]**2)/(shape[0]*shape[1])+\
        np.sum(s_a[s_a<tau*np.max(s_a)]**2)/(shape_a[0]*shape_a[1])
    return c

def cost_error_cut_abs(s,shape,s_a,shape_a):
    c = np.sum(s[s<eps]**2)+np.sum(s_a[s_a<eps]**2)
    return c

eps_max = np.linalg.svd(output_mat[int(np.floor(output_mat.shape[0]/2)):,:int(np.floor(output_mat.shape[1]/2))],compute_uv=False)[0]
print(eps_max)
eps = eps_max*0.5

sys_move,input_dims,output_dims = move.move(system,3,cost_error_cut_rel,n_in=1,n_out=15)
#utils.show_system(sys_move)

print(input_dims)
print(output_dims)

In [ ]:
utils.check_dims(sys_move)

In [ ]:
for i in range(input_dims.shape[0]):
    plt.plot(np.sum(input_dims[:i+1,:],axis=0),"+-")

In [ ]:
for i in range(output_dims.shape[0]):
    plt.plot(np.sum(output_dims[:i+1,:],axis=0),"+-")

In [ ]:
print(sys_move)